# Calendar.csv Cleaning

In [1]:
import pandas as pd
import numpy as np

In [2]:
calendar_oiginal = pd.read_csv('data/calendar.csv')
calendar = calendar_oiginal.copy()

In [3]:
calendar.shape

(1308890, 4)

In [6]:
#this code removes any listings that have no price data.
#remove = calendar.groupby('listing_id').filter(lambda x: x.price.isnull().all())
#cleaned_calendar = calendar.drop(remove.index)
#calendar = cleaned_calendar.copy()
calendar.shape

(1308890, 4)

In [7]:
pd.concat([calendar.iloc[0:2],calendar.dropna().iloc[0:2]])

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
365,3075044,2017-08-22,t,$65.00
366,3075044,2017-08-21,t,$65.00


In [8]:
pd.DataFrame(calendar.dropna().iloc[0].apply(type)).T

,listing_id,date,available,price
365,<class 'numpy.int64'>,<class 'str'>,<class 'str'>,<class 'str'>


In [9]:
calendar.available.replace({'f':False,'t':True}, inplace=True)

In [10]:
test_listing = calendar[calendar['listing_id'] == 3075044]
nadates = list(test_listing[test_listing.price.isnull()].date)

In [12]:
calendar.price = calendar.price.replace('[^0-9.]+','',regex=True).astype(float)

calendar.date = pd.to_datetime(calendar.date)
calendar['day_of_week'] = calendar.date.dt.dayofweek

day_means = calendar.groupby(['listing_id','day_of_week']).price.mean()
day_means = pd.DataFrame(day_means)
day_means.price = day_means.price.fillna(0)
day_means = day_means.to_dict('index')

calendar['price_model'] = list(zip(calendar.listing_id, calendar.day_of_week))
calendar.price_model = calendar.price_model.apply(lambda x: day_means[x]['price'])
calendar['modeled_price'] = np.where(calendar.price.isnull(),calendar.price_model,calendar.price)

In [25]:
test_listing = calendar[calendar['listing_id'] == 3075044]
test_listing[test_listing.date.isin(nadates)]

,listing_id,date,available,price,day_of_week,price_model,modeled_price
682,3075044,2016-10-08,False,NaN,5,75.0,75.0
683,3075044,2016-10-07,False,NaN,4,75.0,75.0
710,3075044,2016-09-10,False,NaN,5,75.0,75.0
727,3075044,2016-09-08,False,NaN,3,65.0,65.0
728,3075044,2016-09-07,False,NaN,2,65.0,65.0
729,3075044,2016-09-06,False,NaN,1,65.0,65.0


In [15]:
calendar.modeled_price.isnull().any()

False

In [ ]:
calendar[(calendar.price != calendar.modeled_price) & (calendar.price.notnull())]
#this is evidence that people don't adjust their prices seasonally

In [16]:
calendar.modeled_price.describe()

count    1.308890e+06
mean     1.560365e+02
std      1.684869e+02
min      0.000000e+00
25%      5.100000e+01
50%      1.200000e+02
75%      2.250000e+02
max      7.163000e+03
Name: modeled_price, dtype: float64

In [20]:
pd.concat([calendar.iloc[0:2],calendar.dropna().iloc[0:2]])

,listing_id,date,available,price,day_of_week,price_model,modeled_price
0,12147973,2017-09-05,False,NaN,1,0.0,0.0
1,12147973,2017-09-04,False,NaN,0,0.0,0.0
365,3075044,2017-08-22,True,65.0,1,65.0,65.0
366,3075044,2017-08-21,True,65.0,0,65.0,65.0


# All critical code in one box

In [ ]:
calendar_oiginal = pd.read_csv('data/calendar.csv')
calendar = calendar_oiginal.copy()


#this code removes any listings that have no price data. We might want to uncomment it.
#remove = calendar.groupby('listing_id').filter(lambda x: x.price.isnull().all())
#cleaned_calendar = calendar.drop(remove.index)
#calendar = cleaned_calendar.copy()

#
calendar.date = pd.to_datetime(calendar.date)
calendar['day_of_week'] = calendar.date.dt.dayofweek

#
calendar.available.replace({'f':False,'t':True}, inplace=True)

#
calendar.price = calendar.price.replace('[^0-9.]+','',regex=True).astype(float)

day_means = calendar.groupby(['listing_id','day_of_week']).price.mean()
day_means = pd.DataFrame(day_means)
day_means.price = day_means.price.fillna(0)
day_means = day_means.to_dict('index')

calendar['price_model'] = list(zip(calendar.listing_id, calendar.day_of_week))
calendar.price_model = calendar.price_model.apply(lambda x: day_means[x]['price'])
calendar['modeled_price'] = np.where(calendar.price.isnull(),calendar.price_model,calendar.price)